# Tennis Players Pose Detection

## Importing libraries

In [ ]:
import os
import io
import base64
from IPython.display import YouTubeVideo, HTML
from os.path import exists, join, basename, splitext

## Installing OpenPose

In [ ]:
repo_git_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(repo_git_url))[0]
if not exists(project_name):
  # installing new CMake because of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # cloning openpose repository
  !git clone -q --depth 1 $repo_git_url
  !cd openpose/ubuntu && ./install_cmake.sh && ./install_cuda.sh && ./install_cudnn.sh
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # installing system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # installing python dependencies
  !pip install -q youtube-dl
  # building openpose
  !cd openpose && git pull origin master && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc```

/bin/bash: line 0: cd: openpose/ubuntu: No such file or directory
Selecting previously unselected package libgflags2.2.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previous

## Downloading Video

In [ ]:
# saving YouTube video ID
YouTubeID = 'bIVzGAMmv1U'
# opening YouTube video
YouTubeVideo(YouTubeID)

In [ ]:
# deleting previous input video
!rm -rf youtube.mp4
# downloading the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YouTubeID
# cuting the first seconds of the video
!ffmpeg -y -loglevel info -i youtube.mp4 -c copy -t 5 video.mp4

## Detecting Poses

In [ ]:
# deleting previous output video
!rm openpose.avi
# detecting poses on the these first seconds
!cd openpose && ./build/examples/openpose/openpose.bin --video video.mp4 --write_json ./output/ --display 0  --write_video openpose.avi
# converting the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

rm: cannot remove 'openpose.avi': No such file or directory
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...

Error:
VideoCapture (IP camera/video) could not be opened for path: 'video.mp4'. If it is a video path, is the path correct?

Coming from:
- /content/openpose/src/openpose/producer/videoCaptureReader.cpp:VideoCaptureReader():54
- /content/openpose/src/openpose/producer/videoCaptureReader.cpp:VideoCaptureReader():58
- /content/openpose/src/openpose/producer/producer.cpp:createProducer():475
- /content/openpose/include/openpose/wrapper/wrapperAuxiliary.hpp:configureThreadManager():1221
- /content/openpose/include/openpose/wrapper/wrapper.hpp:exec():424
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripp

## Showing Poses

In [ ]:
# reading the video
video_encoded = base64.b64encode(io.open('output.mp4', 'rb').read())
# displaying the video
HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
             <source src="data:video/mp4;base64,{2}" type="video/mp4" />
             </video>'''.format(960, 720, video_encoded.decode('ascii')))

FileNotFoundError: ignored